In [1]:
from dataclasses import dataclass
import math
import numpy as np
import pygmo as pg

pop_size: int = 10

def fitness(x: np.ndarray | int) -> np.ndarray[float] | list[float]:
    obj = 0
    for i in range(3):
        obj += (x[2*i-2]-3)**2 / 1000. - (x[2*i-2]-x[2*i-1]) + math.exp(20.*(x[2*i - 2]-x[2*i-1]))

    ce1 = 4*(x[0]-x[1])**2+x[1]-x[2]**2+x[2]-x[3]**2
    ce2 = 8*x[1]*(x[1]**2-x[0])-2*(1-x[1])+4*(x[1]-x[2])**2+x[0]**2+x[2]-x[3]**2+x[3]-x[4]**2
    ce3 = 8*x[2]*(x[2]**2-x[1])-2*(1-x[2])+4*(x[2]-x[3])**2+x[1]**2-x[0]+x[3]-x[4]**2+x[0]**2+x[4]-x[5]**2
    ce4 = 8*x[3]*(x[3]**2-x[2])-2*(1-x[3])+4*(x[3]-x[4])**2+x[2]**2-x[1]+x[4]-x[5]**2+x[1]**2+x[5]-x[0]
    ci1 = 8*x[4]*(x[4]**2-x[3])-2*(1-x[4])+4*(x[4]-x[5])**2+x[3]**2-x[2]+x[5]+x[2]**2-x[1]
    ci2 = -(8*x[5] * (x[5]**2-x[4])-2*(1-x[5]) +x[4]**2-x[3]+x[3]**2 - x[4])

    return [obj, ce1,ce2,ce3,ce4,ci1,ci2]

def get_bounds() -> tuple[np.ndarray, np.ndarray]:
    return ([-5]*6,[5]*6)

def gradient(x: np.ndarray[float]) -> np.ndarray[float]:
    return pg.estimate_gradient_h(lambda x: fitness(x), x)

@dataclass
class my_minlp:
    def fitness(self, x: np.ndarray | int) -> np.ndarray[float] | list[float]:
        print(x)
        
        return fitness(x)
    
    def get_bounds(self) -> tuple[np.ndarray, np.ndarray]:
        """Method that returns the box-bounds of the problem

        Returns:
            tuple[np.ndarray, np.ndarray]: (lower bounds, upper bounds)
        """
        return ([-5]*6,[5]*6)
    
    def get_nec(self) -> int:
        return 0
    
    def get_nic(self) -> int:
        return 6
    
    def get_nix(self):
        """ Number of integer decision variables """
        return 2

    def get_name(self) -> str:
        """ Problem’s name """
        return "MINLP"
    
prob = pg.problem(my_minlp())
print(prob)

algo = pg.algorithm(pg.gaco(gen=10, ker=pop_size))
print(f"Running {algo.get_name()}")
algo.set_verbosity(1) # regulates both screen and log verbosity

# Initialize population and evolve population
pop = pg.population(prob, size=pop_size)
print(f"Initial population: {pop}\nStarting evolution...")
pop = algo.evolve(pop)

# Extract results of evolution
uda=algo.extract(pg.gaco)
print(f"Completed evolution, best fitness: {pop.champion_f[0]}, \nbest decision vector: {pop.champion_x}")

# print(uda.get_log())
for iter_log in uda.get_log():
    print(iter_log)


Problem name: MINLP
	C++ class name: pybind11::object

	Global dimension:			6
	Integer dimension:			2
	Fitness dimension:			7
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	6
	Tolerances on constraints: [0, 0, 0, 0, 0, ... ]
	Lower bounds: [-5, -5, -5, -5, -5, ... ]
	Upper bounds: [5, 5, 5, 5, 5, ... ]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none

Running GACO: Ant Colony Optimization
[-2.69963102 -2.69963102 -2.69963102 -2.69963102 -2.69963102 -2.69963102]
[1.41049366 1.41049366 1.41049366 1.41049366 1.41049366 1.41049366]
[-2.43331201 -2.43331201 -2.43331201 -2.43331201 -2.43331201 -2.43331201]
[2.08147145 2.08147145 2.08147145 2.08147145 2.08147145 2.08147145]
[-2.92309867 -2.92309867 -2.92309867 -2.92309867 -2.92309867 -2.92309867]
[-1.88006095 -1.88006095 -1.8800609

In [3]:
pop


Problem name: MINLP
	C++ class name: pybind11::object

	Global dimension:			6
	Integer dimension:			2
	Fitness dimension:			7
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	6
	Tolerances on constraints: [0, 0, 0, 0, 0, ... ]
	Lower bounds: [-5, -5, -5, -5, -5, ... ]
	Upper bounds: [5, 5, 5, 5, 5, ... ]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 110

	Thread safety: none

Population size: 10

List of individuals: 
#0:
	ID:			2052863505262281032
	Decision vector:	[-0.001102, -0.0171874, 0.95728, -1.64441, 2, ... ]
	Fitness vector:		[3.02702, -0.00220643, -2.00442, -2.00332, -2.00221, ... ]
#1:
	ID:			5778816147284903880
	Decision vector:	[-0.0802741, -0.159189, 0.129833, -0.821082, 2, ... ]
	Fitness vector:		[3.02846, -0.173436, -2.38323, -2.29651, -2.20979, ... ]
#2:
	ID:			1293690899332615608

# Custom problem

In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
from loguru import logger
import pygmo as pg

from solarmed_optimization import (EnvironmentVariables,
                                   ProblemParameters,
                                   ProblemSamples)
from solarmed_optimization.utils.initialization import problem_initialization
from solarmed_optimization.problems import MinlpProblem

logger.disable("phd_visualizations")

#%% Constants
# Paths definition
output_path: Path = Path("../results")
data_path: Path = Path("../data")
fsm_data_path: Path = Path("../results")
date_str: str = "20230703" # "20230707_20230710" # '20230630' '20230703'
pop_size: int = 10


In [2]:
# Either load parameters from json or create a new instance
with open(output_path / "problem_params.json") as f:
    problem_params = ProblemParameters(**json.load(f))
# problem_params = ProblemParameters()

problem_data = problem_initialization(problem_params=problem_params,
                                      date_str=date_str,
                                      data_path=data_path)

ps: ProblemSamples = problem_data.problem_samples
pp: ProblemParameters = problem_data.problem_params
df: pd.DataFrame = problem_data.df
model = problem_data.model

# df_mods: list[pd.DataFrame] = []
df_hors: list[pd.DataFrame] = []
df_sim: pd.DataFrame = None

# Fill missing data
# df['med_vac_state'] = 2

opt_step_idx: int = 0
# for opt_step_idx in range(0, max_opt_steps):
idx_mod = pp.idx_start
# for opt_step_idx in range(0, max_opt_steps):
hor_span = (idx_mod+1, idx_mod+1+ps.n_evals_mod_in_hor_window)


Authorization required, but no authorization protocol specified


In [3]:
# Optimization step `opt_step_idx`

# 1. Initialize the problem instance
## Intialize model instance
ds = df.iloc[idx_mod]

print("")
print(f"Optimization step {opt_step_idx+1}/{ps.max_opt_steps}")

## Environment variables predictions
ds = df.iloc[hor_span[0]:hor_span[1]]
env_vars: EnvironmentVariables = EnvironmentVariables(
    I=ds['I'].values,
    Tamb=ds['Tamb'].values,
    Tmed_c_in=ds['Tmed_c_in'].values,
    cost_w=np.ones((ps.n_evals_mod_in_hor_window, )) * pp.env_params.cost_w,
    cost_e=np.ones((ps.n_evals_mod_in_hor_window, )) * pp.env_params.cost_e,
)

## Initialize problem
prob = pg.problem(
    MinlpProblem(
        model=model, 
        sample_time_opt=pp.sample_time_opt,
        optim_window_time=pp.optim_window_time,
        env_vars=env_vars,
        dec_var_updates=pp.dec_var_updates,
        fsm_valid_sequences=pp.fsm_valid_sequences,
        fsm_data_path=fsm_data_path
    )
)
print(prob)



Optimization step 1/16
box_bounds_lower=array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object)
box_bounds_upper=array([0.0, 8.928, 8.928, 8.928, 8.928, 8.928, 8.928, 8.928, 8.928, 0, 20,
       20, 20, 20, 20, 0, 0, 0, 48, 48, 48, 0, 0, 0, 9, 9, 9, 0, 0, 0, 75,
       75, 75, 0.0, 0.0, 0.0, 35.182176999999996, 35.251404, 35.320631, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 2, 2,
       2], dtype=object)
box_bounds_lower=array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object)
box_bounds_upper=array([0.0, 8.928, 8.928, 8.928, 8.92

In [4]:
# Initialize population and evolve population
pop = pg.population(prob, size=pop_size)

pop


x=array([ 0. ,  2.6,  0.2,  7.8,  3.8,  8.5,  2.7,  6.4,  3.7,  0. , 10. ,
       14. , 15.3,  5.3,  3.1,  0. ,  0. ,  0. ,  3.8,  1.7, 37.2,  0. ,
        0. ,  0. ,  3.8,  5.5,  0.5,  0. ,  0. ,  0. , 28.3, 54.4, 64.8,
        0. ,  0. ,  0. , 22.8, 19.8,  0.7,  0. ,  1. ,  0. ,  0. ,  1. ,
        1. ,  0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,
        1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ])
np.sum(benefit)=15.033690842832314
x=array([ 0. ,  8.1,  0.1,  1. ,  4.3,  8.8,  3.4,  8.2,  4.9,  0. ,  8.4,
       19.5,  9.7,  2.3,  3.6,  0. ,  0. ,  0. ,  3.4, 33.9,  7.7,  0. ,
        0. ,  0. ,  3.3,  7. ,  4.6,  0. ,  0. ,  0. , 29.6,  9. , 44.8,
        0. ,  0. ,  0. , 19.5, 25.1, 12.6,  0. ,  1. ,  1. ,  1. ,  0. ,
        0. ,  0. ,  0. ,  1. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        1. ,  0. ,  0. ,  1. ,  0. ,  2. ,  0. ,  0. ])
np.sum(benefit)=13.968444936239468
x=array([ 0. ,  1.8,  0.5,  2.8,  1.5,  6.2,  4.4,  7.7,  2.1,  0. , 17.4,
       19

Problem name: SolarMED MINLP problem
	C++ class name: pybind11::object

	Global dimension:			63
	Integer dimension:			24
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [0, 0, 0, 0, 0, ... ]
	Upper bounds: [0, 8.928, 8.928, 8.928, 8.928, ... ]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 10

	Thread safety: none

Extra info:

    -	 Size of decision vector: 63 elements
    -	 Decision variable ids: ('qsf', 'qts_src', 'qmed_s', 'qmed_f', 'Tmed_s_in', 'Tmed_c_out', 'sf_active', 'ts_active', 'med_active', 'med_vac_state')
    -	 Decision variable types: (<class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'float'>, <class 'bool'>, <class 'bool'>, <class 'bool'>, <class 'int'>)
    -	 Number of updates per dec.var along

In [ ]:
udp = prob.extract(MinlpProblem)

udp


MinlpProblem(optim_window_size=10800, env_vars=EnvironmentVariables(Tmed_c_in=array([25.1, 25.1, 25.1, 25.1, 25.1, 25.1, 25.1, 25.1, 25.1, 25.1, 25.1,
       25.2, 25.2, 25.2, 25.2, 25.2, 25.2, 25.2, 25.2, 25.2, 25.3, 25.3,
       25.3, 25.3, 25.3, 25.3, 25.3]), Tamb=array([29.9, 29.6, 29.2, 29.4, 29.7, 29.8, 30. , 30. , 30.1, 30.3, 30.4,
       30.7, 31. , 31.2, 31.4, 31.5, 31.5, 31.6, 31.7, 32. , 32.3, 32.7,
       32.9, 33.1, 33.3, 33.7, 34.2]), I=array([299.1, 321. , 344.5, 363.3, 383. , 404.2, 426.1, 445.1, 464.2,
       483.7, 502.7, 523.3, 542.2, 557.3, 573.8, 589.3, 603.9, 616.9,
       631.8, 645. , 655.3, 669.3, 681.2, 692.5, 703.6, 715.7, 724.9]), cost_w=array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3.]), cost_e=array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1]), wmed_f=None), dec_var_updates=Decisio

In [9]:
np.random.rand(10)


array([0.13, 0.18, 0.95, 0.07, 0.8 , 0.76, 0.06, 0.78, 0.98, 0.33])